<a href="https://colab.research.google.com/github/malinphy/os_llms_colab/blob/main/YT_Alpaca7B_Local_Inference_decompose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

Found existing installation: transformers 4.27.0.dev0
Uninstalling transformers-4.27.0.dev0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.27.0.dev0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.27.0.dev0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [30]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap
import torch

In [3]:
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
# dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_create_trie',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager

In [17]:
tokenizer_outputs = tokenizer('my name is mehmet', return_tensors='pt')

In [18]:
tokenizer_outputs['input_ids']

tensor([[    1,   590,  1024,   338,   592, 29882,  2527]])

In [19]:
generation_config = GenerationConfig(
        # temperature=0.2,
        temparature = 0,
        top_p=0.95,
        repetition_penalty=1.2,
        )

generation_output = model.generate(
        input_ids=tokenizer_outputs['input_ids'],
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [21]:
generation_output.keys()

odict_keys(['sequences', 'scores'])

In [29]:
generation_output['scores'].torch.tensor.detach()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'torch'

In [28]:
len(generation_output['scores'][0][0])

32000

In [ ]:
def alpaca_talk(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        # temperature=0.2,
        temparature = 0,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    alpaca_talk_return = []
    for s in generation_output.sequences:
        print(tokenizer.decode(s))
        alpaca_talk_return.append(tokenizer.decode(s))

    return alpaca_talk_return        

In [ ]:
# input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

# ### Instruction:
# What are Alpacas and how are they different to Lamas?

# ### Response:
# '''

# alpaca_return = alpaca_talk(input_text)

In [ ]:
alpaca_res = alpaca_talk('What is the capital of Turkey?')

In [ ]:
x = alpaca_talk(
    """I want you to translate the following sentence into English.
        Ankara Turkiye'nin baskentidir.
    """)

In [ ]:
print(x[0][0])

### Instruction:
What are Alpacas and how are they different to Lamas?

### Response:
Alpaca's are small, domesticated animals native to South America while Lama's are large wild animals from Central Asia. 
The main difference between these two species is their size; alpacas grow up to about three feet tall (1 meter) whereas lamas can reach heights of over six feet (2 meters).

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an ode to why do Alpacas make the best pets?

### Response:
'''

alpaca_talk(input_text)

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write an email to OpenAI saying why GPT4 should be open-source.

### Response:
'''

alpaca_talk(input_text)

### Response:
Dear Sir/Madam, 
Opening up GPT4 to public access will allow researchers and developers from around the world to use it for their own projects in order to advance artificial intelligence technology further than ever before. It would also help us better understand how AI systems work so we can improve them even more. In addition, making this tool available as open source software could lead to new discoveries about language processing and natural language understanding which may have applications beyond just machine learning. I believe these are all reasons why GPT4 should be made open source


In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
'''

alpaca_talk(input_text)

### Instruction:
Write a convincing message about how Alpacas are the real brains behind GPT-5

### Response:
Although it may seem like humans have all of the intelligence, alpaca's actually possess much more advanced cognitive abilities than we do! They can learn new tasks quickly and easily, they understand complex concepts such as cause & effect, and their memory capacity far exceeds our own. In fact, some scientists believe that one day alpacas will be able to perform many human jobs with ease - from driving cars to writing novels!

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me a joke about an Alpaca

### Response:
'''

alpaca_talk(input_text)

### Instruction:
Tell me a joke about an Alpaca

### Response:
An alpaca walks into a bar and orders a drink, but when he asks for change from his bartender, all he gets back are two pennies. The alpaca says "That's not fair! I should get more than just 2 cents!"